In [1]:
# %%
# --- PATHS ---
# Base directory for your processed editions
# Assumes structure: .../images/Volume1/*.png and .../labels/Volume1/*.txt
IMAGES_ROOT = 'Editions/Padova_1618_Cesare_Ripa'
LABELS_ROOT = 'finetuned_complete/structured_inference_output/Padova_1618_Cesare_Ripa/txt'
OUTPUT_DIR = 'runs/pipeline_output'
BINARY_CACHE_DIR = 'runs/binary_cache'
# Configuration and Models
DATA_YAML_PATH = 'RIPA-ft/data.yaml'
MODEL_WEIGHTS = 'my_finetune_project/run_ladas_1280_l_v14/weights/best.pt' 
KRAKEN_SEG_MODEL = 'models/blla.mlmodel' 
OCR_MODEL_PATH = '10.5281/zenodo.2577963'

In [4]:
import os
import glob
import torch
from PIL import Image
from lxml import etree
from tqdm import tqdm
import concurrent.futures
import traceback

# Modern Kraken Imports
from kraken.lib import models, vgsl
from kraken import blla
from kraken import rpred

# Map YOLO classes to TEI Elements
YOLO_TO_TEI_MAP = {
    'MainZone-Head': ('head', {}),
    'MainZone-P': ('p', {}),
    'MainZone-Lg': ('lg', {}),
    'MainZone-Entry': ('div', {'type': 'entry'}),
    'MainZone-ListItem': ('item', {}),
    'MainZone-Date': ('dateline', {}),
    'MainZone-Signature': ('signed', {}),
    'TitlePageZone': ('titlePage', {}),
    'MainZone-Other': ('ab', {'type': 'MainZone-Other'}),
    'MarginTextZone-Notes': ('note', {'place': 'margin'}),
    'RunningTitleZone': ('fw', {'type': 'header'}),
    'NumberingZone': ('fw', {'type': 'pageNum'}),
    'DEFAULT': ('ab', {})
}

# Zones to SKIP OCR entirely (Performance Win)
SKIP_OCR_CLASSES = {
    'DigitizationArtefactZone', 
    'DropCapitalZone',
    'FigureZone',
    'GraphicZone-Decoration', 
    'GraphicZone-FigDesc',
    'GraphicZone-Part',
    'GraphicZone-TextualContent',
    'StampZone', 
    'StampZone-Sticker'
}

class FastPipeline:
    def __init__(self, image_dir, label_dir, output_dir, rec_model_path=None, seg_model_path=None, batch_size=16):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.output_dir = output_dir
        self.batch_size = batch_size # Note: rpred() API in this version may not support direct batching
        
        self.alto_dir = os.path.join(output_dir, 'alto')
        self.tei_dir = os.path.join(output_dir, 'tei')
        os.makedirs(self.alto_dir, exist_ok=True)
        os.makedirs(self.tei_dir, exist_ok=True)

        # Device setup
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        if self.device == 'cuda':
            torch.backends.cudnn.benchmark = True 
            print(f"🚀 Acceleration: CUDA Enabled")
        else:
            print("⚠️ Warning: Running on CPU. This will be slow.")

        # Load Recognition Model
        if rec_model_path is None:
            raise ValueError("rec_model_path required.")
        print(f"Loading Recognition Model: {rec_model_path}")
        self.rec_model = models.load_any(rec_model_path)
        if hasattr(self.rec_model, 'to'): self.rec_model.to(self.device)
        
        # Load Segmentation Model
        self.seg_model = None
        if seg_model_path and os.path.exists(seg_model_path):
            print(f"Loading Segmentation Model: {seg_model_path}")
            self.seg_model = vgsl.TorchVGSLModel.load_model(seg_model_path)
        else:
            # Try default path, but don't crash if missing (let blla.segment handle it)
            try:
                default_path = os.path.join(os.path.dirname(blla.__file__), 'blla.mlmodel')
                if os.path.exists(default_path):
                    print(f"Using Default Segmentation Model at: {default_path}")
                    self.seg_model = vgsl.TorchVGSLModel.load_model(default_path)
            except Exception as e:
                print(f"Warning: Could not load default segmentation model ({e}). Will use internal Kraken fallback.")
        
        if self.seg_model and hasattr(self.seg_model, 'to'): 
            self.seg_model.to(self.device)

        self.class_names = [
            'DigitizationArtefactZone', 'DropCapitalZone', 'FigureZone', 'GraphicZone', 
            'GraphicZone-Decoration', 'GraphicZone-FigDesc', 'GraphicZone-Head', 
            'GraphicZone-Part', 'GraphicZone-TextualContent', 'MainZone-Continued', 
            'MainZone-Date', 'MainZone-Entry', 'MainZone-Head', 'MainZone-Lg', 
            'MainZone-ListItem', 'MainZone-Other', 'MainZone-P', 'MainZone-Signature', 
            'MainZone-Sp', 'MarginTextZone-ContinuedNotes', 'MarginTextZone-ManuscriptAddendum', 
            'MarginTextZone-Notes', 'NumberingZone', 'QuireMarksZone', 'RunningTitleZone', 
            'StampZone', 'StampZone-Sticker', 'TableZone', 'TitlePageZone', 'TitlePageZone-Index'
        ]
        
        self.io_pool = concurrent.futures.ThreadPoolExecutor(max_workers=2)

    def _parse_yolo(self, txt_path, w, h):
        valid_regions = []
        if not os.path.exists(txt_path): return []
        
        with open(txt_path, 'r') as f:
            lines = f.readlines()
            
        for i, line in enumerate(lines):
            parts = line.strip().split()
            if not parts: continue
            c = int(parts[0])
            class_name = self.class_names[c] if c < len(self.class_names) else f"Unknown_{c}"
            
            if class_name in SKIP_OCR_CLASSES:
                continue
                
            x, y, rw, rh = map(float, parts[1:5])
            abs_w = rw * w
            abs_h = rh * h
            abs_x = (x * w) - (abs_w / 2)
            abs_y = (y * h) - (abs_h / 2)
            
            valid_regions.append({
                'id': i,
                'class': class_name,
                'box': (int(max(0, abs_x)), int(max(0, abs_y)), int(abs_w), int(abs_h))
            })
        return valid_regions

    def process_page(self, img_path, label_path):
        """Run OCR pipeline on a single page."""
        img = Image.open(img_path)
        w, h = img.size
        
        ns = "http://www.loc.gov/standards/alto/ns-v4#"
        root = etree.Element("alto", xmlns=ns)
        layout = etree.SubElement(root, "Layout")
        page = etree.SubElement(layout, "Page", ID="p1", WIDTH=str(w), HEIGHT=str(h))
        printspace = etree.SubElement(page, "PrintSpace")

        regions = self._parse_yolo(label_path, w, h)
        
        for reg in regions:
            x, y, rw, rh = reg['box']
            if rw < 5 or rh < 5: continue 
            
            crop = img.crop((x, y, min(w, x+rw), min(h, y+rh)))
            
            text_block = etree.SubElement(printspace, "TextBlock", 
                                          ID=f"region_{reg['id']}_{reg['class']}",
                                          HPOS=str(x), VPOS=str(y),
                                          WIDTH=str(rw), HEIGHT=str(rh))
            text_block.set("TAGREFS", reg['class'])

            try:
                # Segmentation
                bounds = blla.segment(crop, model=self.seg_model, device=self.device)
                
                # Recognition
                if bounds.lines:
                    # Fix: Removed 'batch' argument for compatibility
                    pred_it = rpred.rpred(self.rec_model, crop, bounds)
                    
                    for record in pred_it:
                        l_x1, l_y1, l_x2, l_y2 = 0, 0, 0, 0
                        
                        if hasattr(record, 'boundary') and record.boundary:
                            xs = [pt[0] for pt in record.boundary]
                            ys = [pt[1] for pt in record.boundary]
                            l_x1, l_x2 = min(xs), max(xs)
                            l_y1, l_y2 = min(ys), max(ys)
                        elif hasattr(record, 'baseline') and record.baseline:
                            xs = [pt[0] for pt in record.baseline]
                            ys = [pt[1] for pt in record.baseline]
                            l_x1, l_x2 = min(xs), max(xs)
                            l_y1, l_y2 = min(ys), max(ys)
                        
                        text_line = etree.SubElement(text_block, "TextLine",
                                                     HPOS=str(x + l_x1), VPOS=str(y + l_y1),
                                                     WIDTH=str(l_x2 - l_x1), HEIGHT=str(l_y2 - l_y1))
                        
                        if hasattr(record, 'baseline') and record.baseline:
                            pts = [f"{int(pt[0] + x)},{int(pt[1] + y)}" for pt in record.baseline]
                            etree.SubElement(text_line, "Baseline", POINTS=" ".join(pts))

                        etree.SubElement(text_line, "String", CONTENT=record.prediction)
            except Exception as e:
                # Verbose error printing for debugging
                print(f"⚠️ Error in {os.path.basename(img_path)} | Region {reg['id']}: {e}")
                # traceback.print_exc() 

        return etree.ElementTree(root)

    def _alto_to_tei(self, alto_tree, filename):
        """Converts ALTO tree to TEI Tree using YOLO class mappings."""
        root = alto_tree.getroot()
        tei_ns = "http://www.tei-c.org/ns/1.0"
        xml_ns = "http://www.w3.org/XML/1998/namespace"
        tei_root = etree.Element("TEI", xmlns=tei_ns)
        
        header = etree.SubElement(tei_root, "teiHeader")
        fileDesc = etree.SubElement(header, "fileDesc")
        titleStmt = etree.SubElement(fileDesc, "titleStmt")
        etree.SubElement(titleStmt, "title").text = filename
        
        facsimile = etree.SubElement(tei_root, "facsimile")
        surface = etree.SubElement(facsimile, "surface")
        
        text = etree.SubElement(tei_root, "text")
        body = etree.SubElement(text, "body")
        
        # Namespace-agnostic search for Page/TextBlocks
        page = root.xpath('//*[local-name()="Page"]')[0] if root.xpath('//*[local-name()="Page"]') else None
        if page is not None:
            if 'WIDTH' in page.attrib: surface.set('lrx', page.attrib['WIDTH'])
            if 'HEIGHT' in page.attrib: surface.set('lry', page.attrib['HEIGHT'])

        text_blocks = root.xpath('//*[local-name()="TextBlock"]')
        
        for block in text_blocks:
            tag_ref = block.get("TAGREFS", "DEFAULT")
            block_id = block.get("ID", f"blk_{id(block)}")
            
            hpos = int(float(block.get("HPOS", 0)))
            vpos = int(float(block.get("VPOS", 0)))
            width = int(float(block.get("WIDTH", 0)))
            height = int(float(block.get("HEIGHT", 0)))
            
            # Zone
            etree.SubElement(surface, "zone", {
                f"{{{xml_ns}}}id": f"facs_{block_id}",
                'ulx': str(hpos), 'uly': str(vpos),
                'lrx': str(hpos + width), 'lry': str(vpos + height)
            })
            
            # Semantic Element
            tag_name, tag_attrs = YOLO_TO_TEI_MAP.get(tag_ref, YOLO_TO_TEI_MAP['DEFAULT'])
            elem_attrs = tag_attrs.copy()
            elem_attrs['facs'] = f"#facs_{block_id}"
            semantic_elem = etree.SubElement(body, tag_name, elem_attrs)
            
            # Content
            lines = block.xpath('.//*[local-name()="TextLine"]')
            for line in lines:
                strings = line.xpath('.//*[local-name()="String"]')
                line_text = " ".join([s.get("CONTENT", "") for s in strings])
                lb = etree.SubElement(semantic_elem, "lb")
                lb.tail = f"{line_text} "
                
        return etree.ElementTree(tei_root)

    def save_results(self, alto_tree, basename):
        """Saves ALTO and TEI results."""
        name_no_ext = os.path.splitext(basename)[0]
        try:
            # Save ALTO
            alto_out = os.path.join(self.alto_dir, name_no_ext + ".xml")
            alto_tree.write(alto_out, pretty_print=True, encoding='utf-8', xml_declaration=True)
            
            # Save TEI
            tei_tree = self._alto_to_tei(alto_tree, basename)
            tei_out = os.path.join(self.tei_dir, name_no_ext + ".xml")
            tei_tree.write(tei_out, pretty_print=True, encoding='utf-8', xml_declaration=True)
        except Exception as e:
            print(f"🔥 IO Error for {basename}: {e}")
            traceback.print_exc()

    def process(self):
        images = glob.glob(os.path.join(self.image_dir, "*.[jp][pn]g"))
        print(f"Processing {len(images)} images...")
        
        futures = []
        for img_path in tqdm(images):
            basename = os.path.basename(img_path)
            name_no_ext = os.path.splitext(basename)[0]
            label_path = os.path.join(self.label_dir, name_no_ext + ".txt")
            
            alto_tree = self.process_page(img_path, label_path)
            f = self.io_pool.submit(self.save_results, alto_tree, basename)
            futures.append(f)
        
        for f in concurrent.futures.as_completed(futures):
            try:
                f.result()
            except Exception as e:
                print(f"Thread Exception: {e}")
                
        self.io_pool.shutdown(wait=True)
        print("Done.")



In [5]:
pipeline = FastPipeline(IMAGES_ROOT, LABELS_ROOT, "runs/pipeline_output_new", "models/catmus-print-fondue-large.mlmodel", "models/blla.mlmodel")
pipeline.process()

🚀 Acceleration: CUDA Enabled
Loading Recognition Model: models/catmus-print-fondue-large.mlmodel
Loading Segmentation Model: models/blla.mlmodel
Processing 704 images...


  1%|          | 4/704 [00:30<1:28:43,  7.61s/it]Polygonizer failed on line 0: index 3 is out of bounds for axis 0 with size 0
c:\Users\lucia\anaconda3\Lib\site-packages\kraken\rpred.py:335: UserWarning: Using legacy polygon extractor, as the model was not trained with the new method. Please retrain your model to get speed improvement.
  warnings.warn('Using legacy polygon extractor, as the model was not trained with the new method. Please retrain your model to get speed improvement.')
c:\Users\lucia\anaconda3\Lib\site-packages\kraken\rpred.py:335: UserWarning: Using legacy polygon extractor, as the model was not trained with the new method. Please retrain your model to get speed improvement.
  warnings.warn('Using legacy polygon extractor, as the model was not trained with the new method. Please retrain your model to get speed improvement.')
c:\Users\lucia\anaconda3\Lib\site-packages\kraken\rpred.py:335: UserWarning: Using legacy polygon extractor, as the model was not trained with th

KeyboardInterrupt: 

In [26]:
# %% [markdown]
# # Kraken-Native Document Pipeline (Advanced Configuration)
# 
# **Strategy:** Use Kraken's built-in Masking and Serialization features with fine-tuned parameters for historical documents.
# 
# **References:**
# * [Kraken API Docs](https://kraken.re/main/api_docs.html)
# * [Binarization (nlbin)](https://kraken.re/main/api_docs.html#kraken.binarization.nlbin)
# * [Segmentation (blla)](https://kraken.re/main/api_docs.html#kraken.blla.segment)
# * [Recognition (rpred)](https://kraken.re/main/api_docs.html#kraken.rpred.rpred)
# 
# **Steps:**
# 1.  **Masking:** Convert YOLO coordinates into a binary mask image.
# 2.  **Binarization:** Apply adaptive thresholding (nlbin) with historical-specific params.
# 3.  **Segmentation:** Run BLLA on the full page, restricted by the mask.
# 4.  **OCR:** Run Recognizer on the segmented lines with padding.
# 5.  **Serialization:** Export valid ALTO/XML.

# %%
import os
import glob
import torch
import yaml
import numpy as np
from PIL import Image, ImageDraw
from tqdm.auto import tqdm

# Official Kraken APIs
from kraken.lib import vgsl, models
from kraken.blla import segment as blla_segment
from kraken.rpred import rpred
from kraken.binarization import nlbin
from kraken import serialization

# %% [markdown]
# ## 1. Configuration & Parameters
# Tunable parameters based on Kraken documentation.

# --- ADVANCED PARAMETERS (Docs: https://kraken.re/main/api_docs.html) ---

# Binarization (nlbin)
# zoom: Scale factor for background estimation. 0.5 is faster, 1.0 is more accurate.
# border: Ignore border pixels to avoid artifacts.
BINARIZATION_ARGS = {
    'zoom': 1.0, 
    'border': 0.1, 
    'perc': 80, 
    'range': 20, 
    'low': 5, 
    'high': 90
}

# Segmentation (blla.segment)
# text_direction: 'horizontal-lr' (typical), 'vertical-rl' (Asian scripts), etc.
# padding: Padding added to line bounding boxes.
SEGMENTATION_ARGS = {
    'text_direction': 'horizontal-lr',
    'pad': 0,  # Padding during segmentation phase
}

# Recognition (rpred)
# pad: Extra pixels around the line polygon to include in recognition (crucial for ascenders/descenders)
# bidi_reordering: False for Latin scripts (saves compute), True for Hebrew/Arabic
RECOGNITION_ARGS = {
    'pad': 16,  
    'bidi_reordering': False 
}

# --- CLASS MAPPING ---
DATA_YAML_PATH = 'RIPA-ft/data.yaml'

def load_class_mapping(yaml_path):
    if not os.path.exists(yaml_path): return {0: 'MainZone'}
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)
    return {i: name for i, name in enumerate(data['names'])}

YOLO_CLASSES = load_class_mapping(DATA_YAML_PATH)
print(f"Loaded {len(YOLO_CLASSES)} classes.")

# %% [markdown]
# ## 2. The "Masking" Technique
# Creates a binary mask to restrict Kraken's attention to specific YOLO zones.

# %%
def create_yolo_mask(image_size, txt_path):
    """
    Creates a binary mask (mode '1') where YOLO regions are White (255) 
    and background is Black (0).
    """
    w, h = image_size
    # '1' mode: 0=Black (Ignore), 1=White (Process)
    mask = Image.new('1', (w, h), color=0)
    draw = ImageDraw.Draw(mask)
    
    if not os.path.exists(txt_path):
        return None 

    with open(txt_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5: continue
            
            cls_id = int(parts[0])
            label = YOLO_CLASSES.get(cls_id, 'Unknown')
            
            # Filter: Only text zones. 
            # Skipping purely graphical zones prevents Kraken from hallucinating text in engravings.
            text_keywords = ['Main', 'Margin', 'Title', 'Table', 'Running', 'Numbering', 'Quire']
            is_text = any(x in label for x in text_keywords)
            
            if not is_text: continue

            # Parse Coords (YOLO Center-Normalized -> Pixel Box)
            cx, cy, bw, bh = map(float, parts[1:5])
            x1 = int((cx - bw/2) * w)
            y1 = int((cy - bh/2) * h)
            x2 = int((cx + bw/2) * w)
            y2 = int((cy + bh/2) * h)
            
            # Draw White Rectangle (Active Area)
            draw.rectangle([x1, y1, x2, y2], fill=1)
            
    return mask

# %% [markdown]
# ## 3. Main Processing Function
# Orchestrates the pipeline using the Native API.

# %%
def process_page(image_path, txt_path, seg_model, ocr_model, device):
    # 1. Load & Preprocess
    try:
        im = Image.open(image_path)
        im_rgb = im.convert('RGB') 
    except: return

    # 2. Create Mask
    mask = create_yolo_mask(im.size, txt_path)
    if mask is None: return 

    # 3. Binarize (nlbin)
    try:
        im_bin = nlbin(im_rgb, **BINARIZATION_ARGS)
    except Exception as e:
        print(f"Warning: nlbin failed ({e}), falling back to simple threshold.")
        im_bin = im_rgb.convert('1')

    # 4. Segmentation (Masked)
    with torch.no_grad():
        res = blla_segment(
            im_rgb, 
            model=seg_model, 
            device=device, 
            mask=mask,
            **SEGMENTATION_ARGS
        )

    lines = getattr(res, 'lines', [])
    if not lines:
        return

    # 5. Recognition (Batched)
    with torch.no_grad():
        pred_it = rpred(ocr_model, im_bin, res, **RECOGNITION_ARGS)
        records = list(pred_it)

    # 6. Serialization (Native ALTO)
    # We export ALTO first. This contains the coordinates and text but lacks "Zones".
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    
    alto_out = serialization.serialize(
        records, 
        image_name=os.path.basename(image_path),
        image_size=im.size,
        template='alto' 
    )
    
    alto_path = os.path.join(OUTPUT_DIR, 'alto', f"{base_name}.xml")
    with open(alto_path, 'w', encoding='utf-8') as f:
        f.write(alto_out)
        
    # 7. Semantic Upgrade (Custom TEI)
    # We assume a separate function will handle the YOLO merging
    # This keeps the "Native" pipeline pure but allows custom output
    from lxml import etree as ET
    
    # Load the ALTO we just wrote
    alto_tree = ET.parse(alto_path)
    
    # Run your custom conversion logic (Defined below)
    tei_tree = semantic_alto_to_tei(alto_tree, txt_path, os.path.basename(image_path))
    
    # Write Rich TEI
    tei_path = os.path.join(OUTPUT_DIR, 'tei', f"{base_name}.xml")
    tei_tree.write(tei_path, encoding='UTF-8', xml_declaration=True, pretty_print=True)

# %% [markdown]
# ## 4. Semantic Mapping Logic (Restored)
# This function reads the raw YOLO `.txt` file and maps the Kraken lines to your SemgOnto zones based on geometry.

# %%
def semantic_alto_to_tei(alto_tree, yolo_txt_path, image_filename):
    """
    Matches Kraken ALTO lines to YOLO zones by checking if the line center
    falls inside a YOLO box.
    """
    NS_ALTO = {'a': 'http://www.loc.gov/standards/alto/ns-v4#'}
    NS_TEI = 'http://www.tei-c.org/ns/1.0'
    XML_NS = "{http://www.w3.org/XML/1998/namespace}"
    
    root = ET.Element(f'{{{NS_TEI}}}TEI', nsmap={None: NS_TEI})
    
    # --- Header ---
    header = ET.SubElement(root, 'teiHeader')
    file_desc = ET.SubElement(header, 'fileDesc')
    title_stmt = ET.SubElement(file_desc, 'titleStmt')
    ET.SubElement(title_stmt, 'title').text = f"Transcription of {image_filename}"
    
    # --- Parse YOLO Zones ---
    # We read the zones back into memory to "catch" the lines
    page_el = alto_tree.find('.//a:Page', NS_ALTO)
    h_page = int(float(page_el.get('HEIGHT')))
    w_page = int(float(page_el.get('WIDTH')))
    
    zones = []
    if os.path.exists(yolo_txt_path):
        with open(yolo_txt_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 5: continue
                cls_id = int(parts[0])
                cx, cy, w, h = map(float, parts[1:5])
                
                # Convert to pixels
                x1 = (cx - w/2) * w_page
                y1 = (cy - h/2) * h_page
                x2 = (cx + w/2) * w_page
                y2 = (cy + h/2) * h_page
                
                label = YOLO_CLASSES.get(cls_id, 'MainZone')
                zones.append({'label': label, 'box': (x1, y1, x2, y2)})

    # --- Text Body ---
    text = ET.SubElement(root, 'text')
    body = ET.SubElement(text, 'body')
    div = ET.SubElement(body, 'div', type="page")
    
    # Group ALTO lines by YOLO Zone
    # We iterate through lines, find which zone they belong to, and group them.
    
    # 1. Collect all lines
    lines = alto_tree.findall('.//a:TextLine', NS_ALTO)
    
    # Bucket lines into zones
    zone_content = {i: [] for i in range(len(zones))}
    orphan_lines = []
    
    for line in lines:
        # Calculate line center
        vpos = float(line.get('VPOS'))
        hpos = float(line.get('HPOS'))
        height = float(line.get('HEIGHT'))
        width = float(line.get('WIDTH'))
        cy = vpos + height/2
        cx = hpos + width/2
        
        # Find matching zone
        matched = False
        for i, z in enumerate(zones):
            x1, y1, x2, y2 = z['box']
            if x1 <= cx <= x2 and y1 <= cy <= y2:
                zone_content[i].append(line)
                matched = True
                break
        
        if not matched:
            orphan_lines.append(line)
            
    # 2. Build TEI Structure from Zones
    for i, z in enumerate(zones):
        assigned_lines = zone_content[i]
        if not assigned_lines: continue # Skip empty zones
        
        zone_type = z['label']
        
        # Map to TEI elements
        if 'RunningTitle' in zone_type: el_name, attrs = 'fw', {'type': 'header', 'place': 'top'}
        elif 'Numbering' in zone_type: el_name, attrs = 'fw', {'type': 'page-number', 'place': 'top'}
        elif 'QuireMarks' in zone_type: el_name, attrs = 'fw', {'type': 'signature', 'place': 'bottom'}
        elif 'MarginTextZone' in zone_type:
            subtype = zone_type.split('-')[-1] if '-' in zone_type else 'margin'
            el_name, attrs = 'note', {'place': 'margin', 'type': subtype}
        elif 'MainZone-Head' in zone_type: el_name, attrs = 'head', {}
        elif 'Graphic' in zone_type or 'Figure' in zone_type: el_name, attrs = 'figure', {'type': zone_type}
        else: el_name, attrs = 'ab', {'type': zone_type}
        
        block_el = ET.SubElement(div, el_name, **attrs)
        
        for al_line in assigned_lines:
            lb = ET.SubElement(block_el, 'lb')
            # Extract text
            st = al_line.find('.//a:String', NS_ALTO)
            if st is not None:
                lb.tail = st.get('CONTENT')
                
    # Handle orphans (lines found by Kraken but outside strict YOLO boxes)
    if orphan_lines:
        orphan_block = ET.SubElement(div, 'ab', type="orphan")
        for al_line in orphan_lines:
            lb = ET.SubElement(orphan_block, 'lb')
            st = al_line.find('.//a:String', NS_ALTO)
            if st is not None:
                lb.tail = st.get('CONTENT')

    return ET.ElementTree(root)

# %% [markdown]
# ## 5. Execution Loop

# %%
def run_native_pipeline():
    # Setup
    os.makedirs(os.path.join(OUTPUT_DIR, 'alto'), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, 'tei'), exist_ok=True)
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Running on {device}")
    
    # Load Models
    print("Loading models...")
    seg_model = models.load_any(SEG_MODEL_PATH)
    seg_model.to(device).eval()
    
    ocr_model = models.load_any(OCR_MODEL_PATH)
    ocr_model.to(device).eval()
    
    # Find Data
    images = glob.glob(os.path.join(IMAGES_ROOT, "**", "*.png"), recursive=True)
    images += glob.glob(os.path.join(IMAGES_ROOT, "**", "*.jpg"), recursive=True)
    
    for img_path in tqdm(images):
        # Find label
        rel_path = os.path.relpath(img_path, IMAGES_ROOT)
        txt_name = os.path.splitext(rel_path)[0] + ".txt"
        txt_path = os.path.join(LABELS_ROOT, txt_name)
        
        if os.path.exists(txt_path):
            process_page(img_path, txt_path, seg_model, ocr_model, device)

# Uncomment to run
# run_native_pipeline()

Loaded 30 classes.


In [27]:
run_native_pipeline()

Running on cuda
Loading models...


ValueError: Models of type segmentation are not supported by TorchSeqRecognizer

In [12]:
def run_full_pipeline():
    print("--- Starting Pipeline ---")
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"   Running on: {device}")
    
    # OPTIONAL: Run binarization first
    # precompute_binarization()
    
    try:
        seg_model = vgsl.TorchVGSLModel.load_model(KRAKEN_SEG_MODEL)
        seg_model.to(device)
        ocr_model = vgsl.TorchVGSLModel.load_model(OCR_MODEL_PATH)
        ocr_model.to(device)
        seg_model.eval()
        ocr_model.eval()
    except Exception as e:
        print(f"CRITICAL ERROR: Could not load models. {e}")
        return

    image_files = glob.glob(os.path.join(IMAGES_ROOT, "**", "*.png"), recursive=True)
    image_files += glob.glob(os.path.join(IMAGES_ROOT, "**", "*.jpg"), recursive=True)
    
    alto_dir = os.path.join(OUTPUT_DIR, 'alto')
    tei_dir = os.path.join(OUTPUT_DIR, 'tei')
    os.makedirs(alto_dir, exist_ok=True)
    os.makedirs(tei_dir, exist_ok=True)
    
    print(f"Found {len(image_files)} images.")
    
    for img_path in tqdm(image_files, desc="Processing Pages"):
        rel_path = os.path.relpath(img_path, IMAGES_ROOT)
        rel_txt_path = os.path.splitext(rel_path)[0] + ".txt"
        txt_path = os.path.join(LABELS_ROOT, rel_txt_path)
        
        flat_name = rel_path.replace(os.sep, "_")
        base_name = os.path.splitext(flat_name)[0]
        
        xml_path = os.path.join(alto_dir, f"{base_name}.xml")
        tei_path = os.path.join(tei_dir, f"{base_name}_tei.xml")
        
        if not os.path.exists(txt_path):
            continue
            
        process_single_page(img_path, txt_path, xml_path, tei_path, seg_model, ocr_model, device)
        
    print("--- Complete! ---")

# Uncomment to run
# run_full_pipeline()

In [13]:
import gc  # ADDED: For memory cleanup


In [16]:
import gc
# Kraken imports
from kraken.lib import vgsl
from kraken.lib import models as kraken_models # BETTER LOADER
from kraken.blla import segment as blla_segment
from kraken.rpred import rpred
from kraken.binarization import nlbin 

In [ ]:
!kraken list

^C



Retrieving model list --------------------------------   0% 0/0 -:--:-- 0:00:00
Retrieving model list --------------------------------   0% 0/0 -:--:-- 0:00:01
Retrieving model list -------------------------------   0% 0/65 -:--:-- 0:00:02
Retrieving model list -------------------------------   0% 0/65 -:--:-- 0:00:03
Retrieving model list -------------------------------   0% 0/65 -:--:-- 0:00:04
Retrieving model list -------------------------------   0% 0/65 -:--:-- 0:00:05
Retrieving model list -------------------------------   0% 0/65 -:--:-- 0:00:06
Retrieving model list -------------------------------   0% 0/65 -:--:-- 0:00:07
Retrieving model list -------------------------------   0% 0/65 -:--:-- 0:00:08
Retrieving model list -------------------------------   0% 0/65 -:--:-- 0:00:09
Retrieving model list -------------------------------   0% 0/65 -:--:-- 0:00:10
Retrieving model list -------------------------------   0% 0/65 -:--:-- 0:00:11
Retrieving model list -----------------

In [ ]:

# %%
def sanity_check_ocr():
    print("--- OCR Sanity Check ---")
    
    # 1. AGGRESSIVE MEMORY CLEANUP
    if 'ocr_model' in globals(): del globals()['ocr_model']
    if 'seg_model' in globals(): del globals()['seg_model']
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print(f"   VRAM cleared. Allocated: {torch.cuda.memory_allocated()/1024**2:.2f} MB")

    # Define container class locally to ensure compatibility
    class SanityContainer:
        def __init__(self):
            self.type = 'baselines'
            self.imagename = 'sanity_check.png'
            self.lines = []
            self.regions = {}
            self.script_detection = False # The attribute that caused the crash

    try:
        # 2. Load Model
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"   Loading model on {device}...")
        ocr_model = kraken_models.load_any('10.5281/zenodo.2577963')
        ocr_model.to(device)
        ocr_model.eval()
        
        # 3. Create Synthetic Image
        from PIL import ImageDraw, ImageFont
        img = Image.new('L', (500, 100), color=255)
        draw = ImageDraw.Draw(img)
        
        try:
            # Try to load default font
            # font = ImageFont.load_default() # Sometimes fails to render readable text
            draw.text((10, 40), "Test Transcription 123", fill=0)
        except:
            draw.line((10, 50, 490, 50), fill=0, width=5)
            
        # 4. Binarize & Convert (Mimic Pipeline)
        img_bin = img.convert('1')
        img_input = img_bin.convert('L')
        
        print("   Running inference...")
        with torch.no_grad():
            # Construct the Object Container instead of Dict
            bounds = SanityContainer()
            bounds.lines = [{
                'boundary': [(0,0), (500,0), (500,100), (0,100)],
                'baseline': [(10,80), (490,80)],
                'text_direction': 'horizontal-lr',
                'script': 'default'
            }]
            
            pred_it = rpred(ocr_model, img_input, bounds)
            for record in pred_it:
                print(f"✅ SUCCESS! Prediction: '{record.prediction}'")
                
                del ocr_model
                torch.cuda.empty_cache()
                return

    except Exception as e:
        print(f"❌ FATAL: Sanity check failed. {e}")
        if device == 'cuda':
            print("   Tip: Try restarting the kernel to fully clear GPU memory.")

In [20]:
sanity_check_ocr()

--- OCR Sanity Check ---
   VRAM cleared. Allocated: 0.00 MB
   Loading model on cuda...
❌ FATAL: Sanity check failed. File c:\Users\lucia\Documents\GitHub\Layout-Analysis-and-OCR\10.5281\f.2577963 not loadable by any parser.
   Tip: Try restarting the kernel to fully clear GPU memory.


In [23]:
run_full_pipeline()

--- Starting Pipeline ---
   Running on: cuda
Found 704 images.


Processing Pages:   0%|          | 0/704 [00:00<?, ?it/s]

Page page_0001.png Stats: {'regions': 1, 'lines_found': 0, 'lines_valid': 0, 'text_generated': 0}
Page page_0002.png Stats: {'regions': 0, 'lines_found': 0, 'lines_valid': 0, 'text_generated': 0}
Page page_0003.png Stats: {'regions': 0, 'lines_found': 0, 'lines_valid': 0, 'text_generated': 0}
Page page_0004.png Stats: {'regions': 3, 'lines_found': 0, 'lines_valid': 0, 'text_generated': 0}


Polygonizer failed on line 0: index 2 is out of bounds for axis 0 with size 0


[ERROR] Region 0 failed: 'MultiParamSequential' object has no attribute 'input'
[ERROR] Region 1 failed: 'MultiParamSequential' object has no attribute 'input'
[ERROR] Region 3 failed: 'MultiParamSequential' object has no attribute 'input'
Page page_0005.png Stats: {'regions': 4, 'lines_found': 50, 'lines_valid': 50, 'text_generated': 0}
Page page_0006.png Stats: {'regions': 1, 'lines_found': 0, 'lines_valid': 0, 'text_generated': 0}
[ERROR] Region 1 failed: 'MultiParamSequential' object has no attribute 'input'
[ERROR] Region 4 failed: 'MultiParamSequential' object has no attribute 'input'
[ERROR] Region 5 failed: 'MultiParamSequential' object has no attribute 'input'
[ERROR] Region 6 failed: 'MultiParamSequential' object has no attribute 'input'
Page page_0007.png Stats: {'regions': 8, 'lines_found': 58, 'lines_valid': 55, 'text_generated': 0}
[ERROR] Region 0 failed: 'MultiParamSequential' object has no attribute 'input'
[ERROR] Region 1 failed: CUDA error: out of memory
CUDA kernel

In [ ]:
# %% [markdown]
# # End-to-End Document Layout Analysis & OCR Pipeline
# 
# **Goal:** Convert raw images and YOLO coordinates into standard archival formats (ALTO & TEI).
# 
# **Workflow:**
# 1.  **Setup:** Load class mappings from `data.yaml` and initialize Kraken models.
# 2.  **Inference (Optional):** Run YOLO to generate coordinate files (if missing).
# 3.  **Processing:** #     * Read YOLO coordinates.
#     * Crop regions.
#     * Run Kraken Line Segmentation & OCR on text regions.
#     * Generate ALTO XML (Physical Layout).
#     * Convert to TEI XML (Logical Structure).

# %%
import os
import glob
import time
import yaml
import torch
from pathlib import Path
from PIL import Image
from lxml import etree as ET
from ultralytics import YOLO

# Kraken imports
from kraken.lib import vgsl
from kraken.blla import segment as blla_segment
from kraken.rpred import rpred

# %% [markdown]
# ## 1. Configuration & Paths
# Define where your data lives and where output should go.



# --- DYNAMIC CLASS MAPPING ---
# Loads your 30 classes from data.yaml to ensure names match exactly
def load_class_mapping(yaml_path):
    if not os.path.exists(yaml_path):
        print(f"Warning: {yaml_path} not found. Using minimal fallback.")
        return {0: 'MainZone'}
        
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)
    # Returns dict: {0: 'DigitizationArtefactZone', 1: 'DropCapitalZone', ...}
    return {i: name for i, name in enumerate(data['names'])}

YOLO_TO_SEGMONTO = load_class_mapping(DATA_YAML_PATH)
print(f"Loaded {len(YOLO_TO_SEGMONTO)} classes.")

# %% [markdown]
# ## 2. TEI Conversion Logic
# Converts the physical ALTO layout into a logical TEI structure (`<sourceDoc>` + `<body>`).

# %%
def alto_to_tei(alto_tree, image_filename):
    NS_ALTO = {'a': 'http://www.loc.gov/standards/alto/ns-v4#'}
    NS_TEI = 'http://www.tei-c.org/ns/1.0'
    
    root = ET.Element(f'{{{NS_TEI}}}TEI', nsmap={None: NS_TEI})
    
    # --- 1. teiHeader ---
    header = ET.SubElement(root, 'teiHeader')
    file_desc = ET.SubElement(header, 'fileDesc')
    title_stmt = ET.SubElement(file_desc, 'titleStmt')
    ET.SubElement(title_stmt, 'title').text = f"Transcription of {image_filename}"
    
    # --- 2. sourceDoc (Facsimile/Physical Layer) ---
    source_doc = ET.SubElement(root, 'sourceDoc')
    surface = ET.SubElement(source_doc, 'surface')
    
    page_el = alto_tree.find('.//a:Page', NS_ALTO)
    if page_el is not None:
        surface.set('lry', page_el.get('HEIGHT', '0'))
        surface.set('lrx', page_el.get('WIDTH', '0'))
    surface.set('xml:id', f"page_{image_filename}")
    
    # Copy ALTO blocks to TEI Zones
    for block in alto_tree.findall('.//a:TextBlock', NS_ALTO):
        zone = ET.SubElement(surface, 'zone')
        zone.set('xml:id', block.get('ID'))
        zone.set('type', block.get('TAGREFS', 'MainZone'))
        
        # Copy coordinates
        for attr in ['HPOS', 'VPOS', 'WIDTH', 'HEIGHT']:
            if block.get(attr): zone.set(attr.lower(), block.get(attr))
        
        # Copy lines
        for line in block.findall('.//a:TextLine', NS_ALTO):
            ET.SubElement(zone, 'line').set('xml:id', line.get('ID'))
    
    # --- 3. Text Body (Logical Layer) ---
    text = ET.SubElement(root, 'text')
    body = ET.SubElement(text, 'body')
    div = ET.SubElement(body, 'div', type="page")
    
    # Map SegmOnto zones to TEI semantic elements
    for block in alto_tree.findall('.//a:TextBlock', NS_ALTO):
        zone_type = block.get('TAGREFS', 'MainZone')
        
        # Mapping Logic based on your schema
        if 'RunningTitle' in zone_type:
            el_name, attrs = 'fw', {'type': 'header', 'place': 'top'}
        elif 'Numbering' in zone_type:
            el_name, attrs = 'fw', {'type': 'page-number', 'place': 'top'}
        elif 'QuireMarks' in zone_type:
            el_name, attrs = 'fw', {'type': 'signature', 'place': 'bottom'}
        elif 'MarginTextZone' in zone_type:
            # Extract 'Notes' from 'MarginTextZone-Notes'
            subtype = zone_type.split('-')[-1] if '-' in zone_type else 'margin'
            el_name, attrs = 'note', {'place': 'margin', 'type': subtype}
        elif 'MainZone-Head' in zone_type:
            el_name, attrs = 'head', {}
        elif 'TitlePage' in zone_type:
            el_name, attrs = 'div', {'type': 'titlepage'}
        elif 'Graphic' in zone_type or 'Figure' in zone_type:
            el_name, attrs = 'figure', {'type': zone_type}
        elif 'Table' in zone_type:
            el_name, attrs = 'table', {}
        elif 'MainZone' in zone_type:
            # Extract 'P' from 'MainZone-P'
            subtype = zone_type.split('-')[-1] if '-' in zone_type else 'main'
            el_name, attrs = 'ab', {'type': subtype}
        else:
            # Fallback for Artifacts, Stickers, etc.
            el_name, attrs = 'ab', {'type': zone_type}
            
        # Create logical block linked to facsimile
        struct_block = ET.SubElement(div, el_name, **attrs)
        struct_block.set('facs', f"#{block.get('ID')}")
        
        # Add lines and Text Content
        for line in block.findall('.//a:TextLine', NS_ALTO):
            lb = ET.SubElement(struct_block, 'lb')
            lb.set('facs', f"#{line.get('ID')}")
            
            # Get OCR content if it exists
            string_el = line.find('.//a:String', NS_ALTO)
            if string_el is not None and string_el.get('CONTENT'):
                lb.tail = string_el.get('CONTENT')
            
    return ET.ElementTree(root)

# %% [markdown]
# ## 3. Page Processing Logic
# The core function that processes one image + text file pair.

# %%
def process_single_page(image_path, txt_path, output_xml, output_tei, seg_model, ocr_model, device):
    # 1. Load Image
    try: im = Image.open(image_path)
    except Exception as e: 
        print(f"Error opening image {image_path}: {e}")
        return
    
    w_img, h_img = im.size
    
    if not os.path.exists(txt_path): 
        print(f"WARNING: Missing label file: {txt_path}")
        return

    # 2. Init ALTO Structure
    NS = {'': 'http://www.loc.gov/standards/alto/ns-v4#'}
    root = ET.Element('alto', xmlns="http://www.loc.gov/standards/alto/ns-v4#")
    page = ET.SubElement(ET.SubElement(root, 'Layout'), 'Page', 
                         WIDTH=str(w_img), HEIGHT=str(h_img), ID="p1")
    print_space = ET.SubElement(page, 'PrintSpace', HPOS="0", VPOS="0", 
                                WIDTH=str(w_img), HEIGHT=str(h_img))

    # 3. Process YOLO Regions
    with open(txt_path, 'r') as f: lines = f.readlines()
    
    for idx, line in enumerate(lines):
        parts = line.strip().split()
        if len(parts) < 5: continue
        
        # Parse YOLO Format (class cx cy w h)
        cls_id = int(parts[0])
        cx, cy, w, h = map(float, parts[1:5])
        
        # Denormalize to Pixels
        abs_w, abs_h = int(w * w_img), int(h * h_img)
        abs_x, abs_y = int((cx * w_img) - (abs_w/2)), int((cy * h_img) - (abs_h/2))
        
        # Clip to image bounds (safety)
        abs_x = max(0, abs_x)
        abs_y = max(0, abs_y)
        abs_w = min(abs_w, w_img - abs_x)
        abs_h = min(abs_h, h_img - abs_y)
        
        if abs_w < 5 or abs_h < 5: continue # Skip tiny noise

        # Get Class Name (Fallback to MainZone if unknown)
        label = YOLO_TO_SEGMONTO.get(cls_id, 'MainZone')
        
        # Create ALTO TextBlock
        block = ET.SubElement(print_space, 'TextBlock', ID=f"region_{idx}",
                              HPOS=str(abs_x), VPOS=str(abs_y),
                              WIDTH=str(abs_w), HEIGHT=str(abs_h),
                              TAGREFS=label)
        
        # 4. Kraken Segmentation & OCR
        # We only run expensive OCR on zones that actually contain text
        text_prefixes = [
            'Main', 'Margin', 'Running', 'Numbering', 'Quire', 
            'Title', 'Table', 'Drop', 'Stamp', 'Figure', 'Graphic'
        ]
        # Check if label starts with any prefix OR is explicitly GraphicZone-TextualContent
        is_text = any(label.startswith(p) for p in text_prefixes) or \
                  label == 'GraphicZone-TextualContent'
        
        if is_text:
            try:
                # Crop the region from the main image
                crop = im.crop((abs_x, abs_y, abs_x+abs_w, abs_y+abs_h))
                
                # A. Segment Lines (BLLA)
                res = blla_segment(crop, model=seg_model, device=device)
                
                # B. Run Recognition (OCR) if lines found
                if res and len(res['lines']) > 0:
                    pred_it = rpred(ocr_model, crop, res)
                    
                    for l_idx, (l_data, ocr_record) in enumerate(zip(res['lines'], pred_it)):
                        # Shift local crop coordinates back to global page coordinates
                        pg_bound = [(p[0]+abs_x, p[1]+abs_y) for p in l_data['boundary']]
                        xs, ys = [p[0] for p in pg_bound], [p[1] for p in pg_bound]
                        lx, ly = min(xs), min(ys)
                        lw, lh = max(xs)-lx, max(ys)-ly
                        
                        text_line = ET.SubElement(block, 'TextLine', ID=f"line_{idx}_{l_idx}",
                                                  HPOS=str(int(lx)), VPOS=str(int(ly)),
                                                  WIDTH=str(int(lw)), HEIGHT=str(int(lh)))
                        
                        # Add OCR Text Content
                        ET.SubElement(text_line, 'String', 
                                      CONTENT=ocr_record.prediction,
                                      HPOS=str(int(lx)), VPOS=str(int(ly)),
                                      WIDTH=str(int(lw)), HEIGHT=str(int(lh)))
            except Exception as e:
                print(f"Warning: Segmentation/OCR failed for region {idx} ({label}) in {os.path.basename(image_path)}: {e}")
                continue

    # 5. Save Results
    ET.ElementTree(root).write(output_xml, encoding='UTF-8', xml_declaration=True, pretty_print=True)
    
    tei_tree = alto_to_tei(root, os.path.basename(image_path))
    tei_tree.write(output_tei, encoding='UTF-8', xml_declaration=True, pretty_print=True)

# %% [markdown]
# ## 4. Main Execution Loop
# Runs the pipeline on your dataset.

# %%
def run_full_pipeline():
    print("--- Starting Pipeline ---")
    print("1. Loading Kraken Models (Seg + OCR)...")
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    try:
        seg_model = vgsl.TorchVGSLModel.load_model(KRAKEN_SEG_MODEL)
        seg_model.to(device)
        ocr_model = vgsl.TorchVGSLModel.load_model(OCR_MODEL_PATH)
        ocr_model.to(device)
    except Exception as e:
        print(f"CRITICAL ERROR: Could not load models. {e}")
        return

    # Find images recursively
    image_files = glob.glob(os.path.join(IMAGES_ROOT, "**", "*.png"), recursive=True)
    image_files += glob.glob(os.path.join(IMAGES_ROOT, "**", "*.jpg"), recursive=True)
    
    # Setup output structure
    alto_dir = os.path.join(OUTPUT_DIR, 'alto')
    tei_dir = os.path.join(OUTPUT_DIR, 'tei')
    os.makedirs(alto_dir, exist_ok=True)
    os.makedirs(tei_dir, exist_ok=True)
    
    print(f"Found {len(image_files)} images in {IMAGES_ROOT}")
    print(f"Results will be saved to {OUTPUT_DIR}")
    
    start_time = time.time()
    
    for img_path in image_files:
        # Logic to find corresponding label file
        # Calculates relative path to preserve subfolders if necessary
        rel_path = os.path.relpath(img_path, IMAGES_ROOT)
        rel_txt_path = os.path.splitext(rel_path)[0] + ".txt"
        
        # Construct absolute path to label
        txt_path = os.path.join(LABELS_ROOT, rel_txt_path)
        
        # Construct output filenames (flattened for simplicity in output folder)
        flat_name = rel_path.replace(os.sep, "_")
        base_name = os.path.splitext(flat_name)[0]
        
        xml_path = os.path.join(alto_dir, f"{base_name}.xml")
        tei_path = os.path.join(tei_dir, f"{base_name}_tei.xml")
        
        # Only process if we have coordinates
        if not os.path.exists(txt_path):
            # print(f"Skipping {base_name} (No coordinates found at {txt_path})")
            continue
            
        print(f"Processing {base_name}...")
        process_single_page(img_path, txt_path, xml_path, tei_path, seg_model, ocr_model, device)
        
    print(f"--- Complete! Total time: {time.time() - start_time:.2f}s ---")

# Uncomment to run
run_full_pipeline()

Loaded 30 classes.
--- Starting Pipeline ---
1. Loading Kraken Models (Seg + OCR)...
Found 704 images in Editions/Padova_1618_Cesare_Ripa
Results will be saved to runs/pipeline_output
Processing page_0001...


ValueError: Invalid attribute name 'xml:id'

In [3]:
pip install "numpy<2" --force-reinstall

  Obtaining dependency information for numpy<2 from https://files.pythonhosted.org/packages/3f/6b/5610004206cf7f8e7ad91c5a85a8c71b2f2f8051a0c0c4d5916b76d6cbb2/numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.0 kB 330.3 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/61.0 kB 650.2 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 651.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/15.8 MB 8.2 MB/s eta 0:00:02
   - -------------------------------------- 0.5/15.8 MB 7.1 MB/s eta 0:00:03
   -- ------------------------------------- 0.9/15.8 MB 10.0 MB/s eta 0:00:02
   --- ------------------------------------ 1.4/15.8 MB 10.2 MB/s eta 0:00:02
   --- ------------------------------------ 1.4/15.8 MB 10.2 MB/s eta 0:00:02
   --- ----------------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\lucia\\anaconda3\\Lib\\site-packages\\~-mpy.libs\\libscipy_openblas64_-9e3e5a4229c1ca39f10dc82bba9e2b2b.dll'
Consider using the `--user` option or check the permissions.

